# Erythroid DPT

In [ ]:
import cellrank as cr
import scanpy as sc

import scipy

import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

import os

In [ ]:
import warnings
warnings.simplefilter("ignore")

In [ ]:
sc.set_figure_params(figsize=(2.5, 5), dpi_save=1200, fontsize=12, frameon=False, color_map='magma')

In [ ]:
os.chdir('/research/peer/fdeckert/FD20200109SPLENO')

# Load data 

In [ ]:
# Pre-processed data 
adata = sc.read_h5ad('data/scRNAseq/object/pp.h5ad')

In [ ]:
# GPCCA
g_ery_wt_baseline_d6 = cr.estimators.GPCCA.read('data/scRNAseq/object/cellrank/g_ery_wt_baseline_d6.pickle')
g_ery_wt_cpg_d6 = cr.estimators.GPCCA.read('data/scRNAseq/object/cellrank/g_ery_wt_cpg_d6.pickle')
g_ery_ifnar_fl = cr.estimators.GPCCA.read('data/scRNAseq/object/cellrank/g_ery_ifnar_fl.pickle')
g_ery_ifnar_fl_lysm_cre = cr.estimators.GPCCA.read('data/scRNAseq/object/cellrank/g_ery_ifnar_fl_lysm_cre.pickle')

In [ ]:
# Fate probabilities 
fate_probabilities = pd.concat(
    
    [
        
        pd.DataFrame(g_ery_wt_baseline_d6.fate_probabilities, columns=g_ery_wt_baseline_d6.fate_probabilities.names.tolist(), index=g_ery_wt_baseline_d6.adata.obs_names).rename(columns={"Erythroblast_2": "Erythroblast"}), 
        pd.DataFrame(g_ery_wt_cpg_d6.fate_probabilities, columns=g_ery_wt_cpg_d6.fate_probabilities.names.tolist(), index=g_ery_wt_cpg_d6.adata.obs_names).rename(columns={"Erythroblast_5": "Erythroblast"}), 
        pd.DataFrame(g_ery_ifnar_fl.fate_probabilities, columns=g_ery_ifnar_fl.fate_probabilities.names.tolist(), index=g_ery_ifnar_fl.adata.obs_names).rename(columns={"Erythroblast_1": "Erythroblast"}), 
        pd.DataFrame(g_ery_ifnar_fl_lysm_cre.fate_probabilities, columns=g_ery_ifnar_fl_lysm_cre.fate_probabilities.names.tolist(), index=g_ery_ifnar_fl_lysm_cre.adata.obs_names).rename(columns={"Erythroblast_1": "Erythroblast"})
    
    ]

)

In [ ]:
adata.obs = adata.obs.merge(fate_probabilities, left_index=True, right_index=True, how='left')

# Subset by erythroid lineage 

In [ ]:
celltype_ery = [

    'MegP', 
    'MEP',
    'Proerythroblast', 
    'Erythroblast'
    
]

In [ ]:
adata = adata[adata.obs.celltype_low.isin(celltype_ery)].copy()

# Diffusion pseudotime (DPT)

## Set root cell

In [ ]:
cell_id = adata.obs.index[(adata.obs['Erythroblast']==min(adata.obs['Erythroblast']))][0]
adata.obs['dpt_s_cell'] = adata.obs_names.isin([cell_id])

In [ ]:
adata = adata[adata.obs.celltype_low.isin(['MEP', 'Proerythroblast', 'Erythroblast']) | adata.obs.dpt_s_cell].copy()

In [ ]:
adata.uns['iroot'] = np.flatnonzero(adata.obs['dpt_s_cell'])[0]

# Re-compute embedding on latent space

In [ ]:
sc.pp.neighbors(adata, n_neighbors=30, use_rep='latent')
sc.tl.umap(adata, min_dist=1)

## Compute DPT

In [ ]:
sc.tl.diffmap(adata, n_comps=15)
sc.tl.dpt(adata, n_dcs=15)

In [ ]:
sc.pl.umap(adata, color=['celltype_low', 'label_main_haemopedia', 'dpt_s_cell', 'dpt_pseudotime', 'pct_counts_HB', 'Erythroblast', 'MegP'], legend_loc='on data', ncols=8, size=100)
sc.pl.umap(adata[adata.obs.facility=='BSF'], color=['celltype_low', 'label_main_haemopedia', 'dpt_s_cell', 'dpt_pseudotime', 'pct_counts_HB', 'Erythroblast', 'MegP'], legend_loc='on data', ncols=8, size=100)
sc.pl.umap(adata[adata.obs.facility=='VBC'], color=['celltype_low', 'label_main_haemopedia', 'dpt_s_cell', 'dpt_pseudotime', 'pct_counts_HB', 'Erythroblast', 'MegP'], legend_loc='on data', ncols=8, size=100)

# Remove starting cell if not MEP 

In [ ]:
adata = adata[adata.obs.celltype_low.isin(['MEP', 'Proerythroblast', 'Erythroblast'])].copy()

# Save results 

In [ ]:
adata.obs[['dpt_pseudotime']].to_csv('result/pseudotime/ery_dpt.csv')